# E-Commerce Shipping Data Preprocessing

이 노트북은 배송 지연 여부를 예측하기 위한 데이터 전처리 및 피처 엔지니어링 과정을 담고 있습니다.

**주요 내용:**
1. 데이터 로드 및 확인
2. 데이터 정제 (불필요 컬럼 삭제)
3. 피처 엔지니어링 (파생 변수 생성)
4. 데이터 전처리 (인코딩, 스케일링)
5. 상관관계 분석

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 한글 폰트 설정 (Windows/Mac)
if os.name == 'nt':
    plt.rc('font', family='Malgun Gothic')
elif os.name == 'posix':
    plt.rc('font', family='AppleGothic')
else:
    print("Unknown OS")

plt.rc('axes', unicode_minus=False) # 마이너스 폰트 깨짐 방지

import warnings
warnings.filterwarnings('ignore')

## 1. 데이터 로드 (Load Data)
데이터셋을 로드하고 기본적인 정보를 확인합니다.

In [ ]:
# 데이터 로드
df = pd.read_csv('data/Train.csv')

# 데이터 확인
print(f"데이터 크기: {df.shape}")
display(df.head())
display(df.info())

## 2. 데이터 정제 (Data Cleaning)
- **ID 삭제**: `ID` 컬럼은 고유 식별자로, 모델 학습에 도움이 되지 않으므로 삭제합니다.
- **타겟 변수 명칭 변경**: `Reached.on.Time_Y.N`은 너무 길고 특수문자가 포함되어 있어 `target`으로 변경합니다.

In [ ]:
# ID 컬럼 삭제
if 'ID' in df.columns:
    df = df.drop('ID', axis=1)
    print("'ID' 컬럼이 삭제되었습니다.")

# 타겟 변수 이름 변경
if 'Reached.on.Time_Y.N' in df.columns:
    df = df.rename(columns={'Reached.on.Time_Y.N': 'target'})
    print("타겟 변수 이름이 'target'으로 변경되었습니다.")

display(df.head())

## 3. 피처 엔지니어링 (Feature Engineering)
데이터의 잠재적인 패턴을 모델이 더 잘 학습할 수 있도록 새로운 파생 변수를 생성합니다.

1. **Discount_Rate (할인율)**: `Discount_offered` / `Cost_of_the_Product`. 제품 가격 대비 할인 비율입니다.
2. **Is_High_Discount (고할인 여부)**: 할인이 10달러를 초과하면 1, 아니면 0. 높은 할인은 배송 지연과 강한 상관관계가 있을 수 있습니다.
3. **Cost_per_Weight (무게당 비용)**: `Cost_of_the_Product` / `Weight_in_gms`.
4. **Weight_Range (무게 구간)**: `Weight_in_gms`를 구간별로 나누어 범주화합니다 (Low, Medium, High, Very High).

In [ ]:
# 1. 할인율 (Discount Rate)
df['Discount_Rate'] = (df['Discount_offered'] / df['Cost_of_the_Product']) * 100

# 2. 고할인 여부 (Is High Discount)
df['Is_High_Discount'] = (df['Discount_offered'] > 10).astype(int)

# 3. 무게당 비용 (Cost per Weight)
df['Cost_per_Weight'] = df['Cost_of_the_Product'] / df['Weight_in_gms']

# 4. 무게 구간화 (Weight Range)
# 0-2000, 2000-4000, 4000-6000, 6000+
bins = [0, 2000, 4000, 6000, float('inf')]
labels = ['Low', 'Medium', 'High', 'Very High']
df['Weight_Range'] = pd.cut(df['Weight_in_gms'], bins=bins, labels=labels)

# 생성된 파생 변수 확인
display(df[['Discount_Rate', 'Is_High_Discount', 'Cost_per_Weight', 'Weight_Range']].head())

## 4. 데이터 전처리 (Preprocessing)
모델링을 위해 수치형 데이터는 스케일링하고, 범주형 데이터는 인코딩합니다.

### 4.1 인코딩 (Encoding)
- **Binary Encoding**: `Gender` (F=0, M=1)
- **Ordinal Encoding**: `Product_importance` (low=0, medium=1, high=2)
- **One-Hot Encoding**: `Warehouse_block`, `Mode_of_Shipment`, `Weight_Range`

In [ ]:
# Gender Mapping
df['Gender'] = df['Gender'].map({'F': 0, 'M': 1})

# Product_importance Mapping
importance_map = {'low': 0, 'medium': 1, 'high': 2}
df['Product_importance'] = df['Product_importance'].map(importance_map)

print("Mapping (Gender, Product_importance) 완료")

# One-Hot Encoding
# Weight_Range도 포함하여 원-핫 인코딩 진행
categorical_cols = ['Warehouse_block', 'Mode_of_Shipment', 'Weight_Range']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=False)

print("One-Hot Encoding 완료")
print(f"최종 컬럼 수: {df.shape[1]}")
display(df.columns)

### 4.2 스케일링 (Scaling)
수치형 변수들의 스케일을 맞추기 위해 `StandardScaler`를 사용합니다.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# 스케일링할 수치형 컬럼 목록
numeric_cols = ['Cost_of_the_Product', 'Weight_in_gms', 'Discount_offered', 
                'Prior_purchases', 'Customer_care_calls', 'Customer_rating',
                'Discount_Rate', 'Cost_per_Weight']

df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

print("StandardScaler 적용 완료")
display(df[numeric_cols].describe())

## 5. 상관관계 분석 (Correlation Analysis)
전처리가 완료된 데이터셋의 상관관계를 시각화하여 변수 간의 관계를 파악합니다.

In [ ]:
plt.figure(figsize=(18, 15))
sns.heatmap(df.corr(), annot=True, fmt='.2f', cmap='coolwarm', linewidths=0.5)
plt.title('전처리 완료된 데이터 상관관계 히트맵 (Feature Correlation Matrix)')
plt.show()

## 6. 결론 및 저장
전처리가 완료된 데이터를 확인하고 필요한 경우 저장합니다.

In [ ]:
display(df.head())

# 필요 시 csv 저장 코드 (주석 처리)
# df.to_csv('data/Train_processed.csv', index=False)
print("전처리 완료")